# Test Cache in Jupyter Notebook

In [1]:
import helpers.cache as hcac

In [2]:
import helpers.dbg as dbg

# Define computation function

This function will be subjected to cache.

In [3]:
def computation_function(a, b):
    # hello
    # assert 0
    out = a * b
    print("Multiplication: %s * %s = %s" % (a, b, out))
    return out


inputs = (1, 2)
exp_output = 2

computation_function(*inputs)

Multiplication: 1 * 2 = 2


2

## Memory cache

In [4]:
memory_cached_computation = hcac.Cached(
    computation_function, use_mem_cache=True, use_disk_cache=False
)

memory_cached_computation.clear_cache()

dbg.dassert_eq(memory_cached_computation(*inputs), exp_output)
dbg.dassert_eq(memory_cached_computation.get_last_cache_accessed(), "no_cache")

dbg.dassert_eq(memory_cached_computation(*inputs), exp_output)
dbg.dassert_eq(memory_cached_computation.get_last_cache_accessed(), "mem")

print("memory caching checks passed")

Skipping disk cache


Multiplication: 1 * 2 = 2
memory caching checks passed


In [5]:
def computation_function(a, b):
    # hello
    # assert 0
    out = a * b
    print("Multiplication: %s * %s = %s" % (a, b, out))
    return out


inputs = (1, 2)
exp_output = 2

dbg.dassert_eq(memory_cached_computation(*inputs), exp_output)
dbg.dassert_eq(memory_cached_computation.get_last_cache_accessed(), "mem")

## Disk cache

In [6]:
disk_cached_computation = hcac.Cached(
    computation_function, use_mem_cache=False, use_disk_cache=True
)

disk_cached_computation.clear_cache("disk")

dbg.dassert_eq(disk_cached_computation(*inputs), exp_output)
dbg.dassert_eq(disk_cached_computation.get_last_cache_accessed(), "no_cache")

dbg.dassert_eq(disk_cached_computation(*inputs), exp_output)
dbg.dassert_eq(disk_cached_computation.get_last_cache_accessed(), "disk")

print("disk caching checks passed")

Skipping memory cache
Skipping memory cache


Multiplication: 1 * 2 = 2
disk caching checks passed


## Full cache

In [7]:
fully_cached_computation = hcac.Cached(
    computation_function, use_mem_cache=True, use_disk_cache=True
)

fully_cached_computation.clear_cache()

dbg.dassert_eq(fully_cached_computation(*inputs), exp_output)
dbg.dassert_eq(fully_cached_computation.get_last_cache_accessed(), "no_cache")

dbg.dassert_eq(fully_cached_computation(*inputs), exp_output)
dbg.dassert_eq(fully_cached_computation.get_last_cache_accessed(), "mem")

dbg.dassert_eq(fully_cached_computation(*inputs), exp_output)
dbg.dassert_eq(fully_cached_computation.get_last_cache_accessed(), "mem")

print("Clear mem cache")
fully_cached_computation.clear_cache("mem")

dbg.dassert_eq(fully_cached_computation(*inputs), exp_output)
dbg.dassert_eq(fully_cached_computation.get_last_cache_accessed(), "disk")

dbg.dassert_eq(fully_cached_computation(*inputs), exp_output)
dbg.dassert_eq(fully_cached_computation.get_last_cache_accessed(), "mem")

print("full caching checks passed")

Multiplication: 1 * 2 = 2
Clear mem cache
full caching checks passed


In [8]:
dbg.dassert_eq(fully_cached_computation(*inputs), exp_output)
dbg.dassert_eq(fully_cached_computation.get_last_cache_accessed(), "mem")

In [9]:
# This should fail all the times, because we clear the memory cache.
fully_cached_computation.clear_cache("mem")
dbg.dassert_eq(fully_cached_computation(*inputs), exp_output)
dbg.dassert_eq(fully_cached_computation.get_last_cache_accessed(), "mem")

AssertionError: 
################################################################################
* Failed assertion *
'disk'
==
'mem'
################################################################################
